## Dice Game Simulation

Playing poker dice suggested a question I couldn't answer using probability techniques from A Level Statistics: what is the expectation value for the number of rolls required it would take to get all sixes from five dice, if at each round sixes are reserved (i.e. without replacement).

The expectation value for the maximum time to first event for five concurrent poisson processes would be easy to calculate, but the same technique cannot be used here.

I've since found how to calculate the answer without having to estimate it from a Monte Carlo simulation so I have added that method as a check/comparison.

I have tried to make the functions as general as possible. First we calculate the transition probability between numbers of sixes, then at each stage of a particular trial use these to calculate cumulative probabilities for all possible events resulting from the next roll. A random number on [0,1] is used to determine which event is carried out. The process is continued until the conclusion of each trial, and repeated for a set number of trials. 

The results from each trial are stored and used to calculate a mean and standard error, which are compared with the answer obtained from calculation via the transition matrix - see https://en.wikipedia.org/wiki/Absorbing_Markov_chain.

In [2]:
import numpy
import scipy.special


def prob(d, psuccess, a, b):
    # transition probabilities from a sixes to b sixes given d dice
    p = psuccess
    q = 1.0 - p

    r = d - a 			# number of dice left to roll
    s = b - a			# number of successes required
    f = d - b			# number of failures required

    C = scipy.special.comb(r, s)    # number of ways to get s sixes from r dice

    if (b < a):
        pi = 0
    elif (b == a):
        pi = (q**f)
    elif (b > a):
        pi = C * (p**s) * (q**f)

    return pi       			# transition probability from a to b sixes


def cp(d, psuccess, a, b):
    # cumulative probability from current number of sixes (a) to b sixes
    cp = 0
    for i in range(a, b+1):
        cp += prob(d, psuccess, a, i)
    return cp


def trial(d, psuccess):
    sixes = 0 			        # number of sixes
    rollcount = 0 			    # number of rolls

    while (sixes < d):
        # run trial until all dice show sixes
        rollcount += 1
        rand = numpy.random.random()
        for i in range(sixes, d+1):
            if rand >= cp(d, psuccess, sixes, i-1):
                temp = i
        sixes = temp

    return rollcount


def simulation(trials, d, psuccess):
    results = []

    for i in range(trials):
        rollcount = trial(d, psuccess)
        results.append(rollcount)
        if i % (trials/10) == 0:
            print 'loading...', 100 * i / trials, '%'

    resultsarray = numpy.array(results)

    mean = numpy.mean(resultsarray)
    sd = numpy.std(resultsarray)
    stderror = sd / (trials**0.5)
    return mean, stderror


def transitionmatrix(d, psuccess):
    Q = numpy.zeros((d, d))
    for i in range(d):
        for j in range(d):
            Q[i, j] = prob(d, psuccess, i, j)
    return Q


def ex_analyticsolution(d, psuccess):
    Q = transitionmatrix(d, psuccess)
    Id = numpy.identity(d)
    N = numpy.linalg.inv(Id - Q)
    onesV = numpy.ones(d)
    EXV = numpy.dot(N, onesV)
    EX = EXV[0]
    return EX


# Parameters
d = 5                   # number of dice
psuccess = (1.0/6.0)    # probability of rolling a six for a single die
trials = 50000          # number of trials

# Main Program
EXA = ex_analyticsolution(d, psuccess)
print '\nI have', d, 'dice. I roll them all, reserving any sixes. How many rolls to get all sixes?'
print '\nAnalytic solution (transition matrix):\nExpectation value for number of rolls required =', EXA, '\n'

print 'Monte Carlo simulation using', trials, 'trials for determining expectation value'

mean, stderror = simulation(trials, d, psuccess)
print 'Simulation complete\n'
print 'Experimental results after', trials, 'trials:'
print 'Expectation value for number of rolls required =', mean, '+/-', stderror, '\n'

if abs(mean - EXA) < (2*stderror):
    print 'This is consistent with the analytic solution.\n'



I have 5 dice. I roll them all, reserving any sixes. How many rolls to get all sixes?

Analytic solution (transition matrix):
Expectation value for number of rolls required = 13.0236615076 

Monte Carlo simulation using 50000 trials for determining expectation value
loading... 0 %
loading... 10 %
loading... 20 %
loading... 30 %
loading... 40 %
loading... 50 %
loading... 60 %
loading... 70 %
loading... 80 %
loading... 90 %
Simulation complete

Experimental results after 50000 trials:
Expectation value for number of rolls required = 13.0475 +/- 0.0298905884017 

This is consistent with the analytic solution.



The analytic solution is pretty much instantaneous - when running the simulation on jupyter.org it takes around 25 seconds for 50,000 trials. As seen above, simulation results are consistent with the solution via the transition matrix - although both use the same method for calculating transition probabilities and so are not independent. Printing this matrix it is possible to check individual probabilities with a calculator and they are as expected.